In [1]:
import importlib
import cnn
import training
importlib.reload(cnn)
importlib.reload(training)
import torch as T
import torch.nn.functional as F
import torch.optim as o
import torch.optim.lr_scheduler as s
from torch.utils.data import DataLoader

In [2]:
device = T.device('cuda' if T.cuda.is_available() else 'cpu')

model = cnn.LetterClassifierCnn()
model.to(device)
print(f"{sum(p.numel() for p in model.parameters() if p.requires_grad)} parameters on '{device}'.")

669642 parameters on 'cuda'.


In [3]:
learning_rate = 0.001
optimizer = o.Adam(model.parameters(), lr=learning_rate)
scheduler = s.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

In [4]:
batch_size = 128
train_dataset = T.load('train.pt')
test_dataset = T.load('test.pt')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f'Training samples: {len(train_dataset)} Testing samples: {len(test_dataset)}')

Training samples: 124800 Testing samples: 20800


In [5]:
for epoch in range(32):
    print(f'Epoch: {epoch+1}')
    loss, accuracy = training.train(model, train_loader, device, optimizer)
    print(f'Training loss: {loss:.4f} accuracy: {accuracy:.2f}%')
    loss, accuracy = training.evaluate(model, test_loader, device)
    print(f'Evaluation loss: {loss:.4f} accuracy: {accuracy:.2f}%')
    scheduler.step(loss)

Epoch: 1
Training loss: 0.4328 accuracy: 86.42%
Evaluation loss: 0.2541 accuracy: 91.64%
Epoch: 2
Training loss: 0.2047 accuracy: 93.13%
Evaluation loss: 0.1997 accuracy: 93.41%
Epoch: 3
Training loss: 0.1644 accuracy: 94.24%
Evaluation loss: 0.1988 accuracy: 93.42%
Epoch: 4
Training loss: 0.1394 accuracy: 94.96%
Evaluation loss: 0.1830 accuracy: 93.96%
Epoch: 5
Training loss: 0.1204 accuracy: 95.48%
Evaluation loss: 0.1927 accuracy: 93.72%
Epoch: 6
Training loss: 0.1053 accuracy: 95.98%
Evaluation loss: 0.1962 accuracy: 93.87%
Epoch: 7
Training loss: 0.0921 accuracy: 96.37%
Evaluation loss: 0.2086 accuracy: 93.72%
Epoch: 8
Training loss: 0.0805 accuracy: 96.79%
Evaluation loss: 0.2198 accuracy: 93.56%
Epoch: 9
Training loss: 0.0528 accuracy: 97.80%
Evaluation loss: 0.2111 accuracy: 94.12%
Epoch: 10
Training loss: 0.0458 accuracy: 98.03%
Evaluation loss: 0.2204 accuracy: 94.20%
Epoch: 11
Training loss: 0.0426 accuracy: 98.18%
Evaluation loss: 0.2271 accuracy: 94.20%
Epoch: 12
Training 

In [12]:
#T.save({'model_state_dict': model.state_dict()}, 'letterClassification.nn');

model.eval()
model.to(T.device('cpu'))
example_input = T.randn(1, 1, 28, 28)  # Adjust the input size as per your model
#example_input = example_input.to(device)
scripted_model = T.jit.trace(model, example_input)

scripted_model.save("model.pt")


In [3]:
from torchvision.datasets import EMNIST
from torchvision.transforms import Compose, ToTensor, Normalize

# Define preprocessing transforms
transform = Compose([
    ToTensor(),  # Convert image to PyTorch tensor
    Normalize((0.5,), (0.5,))  # Normalize pixel values to [-1, 1]
])

# Download and load the dataset
train_dataset = EMNIST(root='../../../../DataSets', split='letters', train=True, download=False, transform=transform)
test_dataset = EMNIST(root='../../../../DataSets', split='letters', train=False, download=False, transform=transform)

T.save(train_dataset, 'train.pt')
T.save(test_dataset, 'test.pt')

In [ ]:
import matplotlib.pyplot as plt

count = 0
for image, label in train_dataset:
    if label != 11:
        continue
    count += 1

    img = image.numpy()

    plt.imshow(img, cmap='gray')
    plt.show()

    if count == 10:
        break;
